In [0]:
pacman::p_load(SparkR, dplyr, labelled, haven, sf, stringr, naniar)

#### Import GADM gpkg

In [0]:
read_path = "/Volumes/idm_dhs/dhs_data/dhs_data_shp/gadm_410.gpkg"

gadm <- st_read(read_path)

str(gadm)

In [0]:
gadm1 <- gadm %>%
  dplyr::mutate(across(c(GID_0, GID_1, GID_2, GID_3, GID_4, GID_5), ~ str_remove_all(.x, pattern = fixed(" ")))) %>%
  dplyr::mutate(across(c(-geom), as.character)) %>%
  dplyr::mutate(across(c(-geom), na_if, "")) %>%
  dplyr::mutate(GID_n = ifelse(is.na(GID_5), GID_4, GID_5),
         GID_n = ifelse(is.na(GID_4), GID_3, GID_n),
         GID_n = ifelse(is.na(GID_3), GID_2, GID_n),
         GID_n = ifelse(is.na(GID_2), GID_1, GID_n),
         GID_n = ifelse(is.na(GID_1), GID_0, GID_n)) %>%
  dplyr::filter(GID_0 == "NGA")

str(gadm1)

#### Import DHS GPS data

In [0]:
shp_files <- c("NGGE23FL", "NGGE4BFL", "NGGE52FL", "NGGE61FL", "NGGE6AFL", "NGGE71FL", "NGGE7BFL", "NGGE81FL")

# READ IN GEOSPATIAL FILES AND BIND TOGETHER TO CREATE A SINGLE FILE
pts_comb <- data.frame()
for (gf in shp_files){
  read_path = paste0("/Volumes/idm_dhs/dhs_data/dhs_data_shp/", gf, "/", gf, ".shp")
  pts <- st_read(read_path, quiet=TRUE) %>%
    st_make_valid() %>%
    dplyr::mutate(geo_file = gf,
                  survey = paste0(substr(gf, 1, 2), substr(gf, nchar(gf) - 3, nchar(gf)))) %>%
    dplyr::mutate(across(c(-geometry), ~ case_when(. == "NULL" ~ NA,
                                                   TRUE ~ .)))
  pts_comb <- rbind(pts_comb, pts)
  print(gf)
}

pts_comb <- pts_comb %>%
  dplyr::mutate(survey = case_when(survey == "NG23FL" ~ "NG21FL",
                                   survey == "NG52FL" ~ "NG53FL",
                                   TRUE ~ survey))

pts_comb1 <- st_join(pts_comb, gadm1, join=st_within) %>%
  dplyr::mutate(cluster_lon = sf::st_coordinates(.)[,1],
                cluster_lat = sf::st_coordinates(.)[,2]) %>%
  st_drop_geometry() %>%
  dplyr::select(survey, geo_file, DHSCC, DHSYEAR, DHSCLUST, GID_0, NAME_1, cluster_lon, cluster_lat) %>%
  distinct() %>%
  setNames(c("survey", "geo_file", "dhs_cc", "survey_year", "v001", "iso_code", "admin1", "cluster_lon", "cluster_lat"))

str(pts_comb1)

### Import DHS PR data

In [0]:
db = "nga"
df_tables <- SparkR::sql(paste0("SHOW TABLES IN idm_dhs.", db)) %>% as.data.frame()

df_tables_pr <- df_tables %>%
  filter(grepl('pr', tableName)) %>%
  mutate(table_name = paste0(database, ".", tableName))

display(df_tables_pr)

In [0]:
raw_dta_folder = "/Volumes/idm_dhs/dhs_data/dhs_data_dta"
file_list <- list()
for (file_name in dbutils.fs.ls(raw_dta_folder)){
  file_list <- append(file_list, file_name$name)
}

df_file_list <- data.frame(file_name = unlist(file_list))

df_file_list <- df_file_list %>%
  filter(startsWith(toupper(file_name), "NGPR"))

display(df_file_list)

In [0]:
chn_indicators <- data.frame()
for (dhs_file in unique(df_file_list$file_name)){
  print(dhs_file)

  file_path = paste0("/Volumes/idm_dhs/dhs_data/dhs_data_dta/", dhs_file)
  PRdata <- read_dta(file_path) %>%
    dplyr::mutate(survey = word(dhs_file, 1, sep = "\\.")) %>%
    dplyr::mutate(survey = toupper(paste0(substr(survey, 1, 2), substr(survey, nchar(survey) - 3, nchar(survey)))))


# /*****************************************************************************************************
# Program: 			NT_CH_NUT.R
# Purpose: 			Code to compute anthropometry and anemia indicators in children
# Data inputs: 	PR dataset
# Data outputs:	coded variables
# Author:				Shireen Assaf
# Date last modified: July 31, 2023 by Shireen Assaf 
# *****************************************************************************************************/
# 
# /*----------------------------------------------------------------------------
# Variables created in this file:
# nt_ch_sev_stunt		"Severely stunted child under 5 years"
# nt_ch_stunt			  "Stunted child under 5 years"
# nt_ch_mean_haz		"Mean z-score for height-for-age for children under 5 years"
# nt_ch_sev_wast		"Severely wasted child under 5 years"
# nt_ch_wast			  "Wasted child under 5 years"
# nt_ch_ovwt_ht		  "Overweight for heigt child under 5 years"
# nt_ch_mean_whz		"Mean z-score for weight-for-height for children under 5 years"
# nt_ch_sev_underwt	"Severely underweight child under 5 years"
# nt_ch_underwt		  "Underweight child under 5 years"
# nt_ch_ovwt_age		"Overweight for age child under 5 years"
# nt_ch_mean_waz		"Mean weight-for-age for children under 5 years"
# 	
# nt_ch_any_anem		"Any anemia - child 6-59 months"
# nt_ch_mild_anem		"Mild anemia - child 6-59 months"
# nt_ch_mod_anem		"Moderate anemia - child 6-59 months"
# nt_ch_sev_anem		"Severe anemia - child 6-59 months"
# ----------------------------------------------------------------------------*/
# 
PRdata <- PRdata %>%
  mutate(wt = hv005/1000000)

# *** Anthropometry indicators ***
if (!"hv103" %in% names(PRdata)){PRdata$hv103 <- NA}
if (!"hc70" %in% names(PRdata)){PRdata$hc70 <- NA}
if (!"hc72" %in% names(PRdata)){PRdata$hc72 <- NA}
if (!"hc71" %in% names(PRdata)){PRdata$hc71 <- NA}
if (!"hc1" %in% names(PRdata)){PRdata$hc1 <- NA}
if (!"hc56" %in% names(PRdata)){PRdata$hc56 <- NA}


# //Severely stunted
PRdata <- PRdata %>%
  mutate(nt_ch_sev_stunt =
           case_when(
             hv103==1 &  hc70 < -300  ~ 1 ,
             hv103==1 &  hc70 >= -300 & hc70 < 9996 ~ 0 ,
             hc70 >= 9996 ~ 99)) %>%
  replace_with_na(replace = list(nt_ch_sev_stunt = c(99))) %>%
  set_value_labels(nt_ch_sev_stunt = c("Yes" = 1, "No"=0  )) %>%
  set_variable_labels(nt_ch_sev_stunt = "Severely stunted child under 5 years")

# //Stunted
PRdata <- PRdata %>%
  mutate(nt_ch_stunt =
           case_when(
             hv103==1 &  hc70 < -200  ~ 1 ,
             hv103==1 &  hc70 >= -200 & hc70 < 9996 ~ 0 ,
             hc70 >= 9996 ~ 99)) %>%
  replace_with_na(replace = list(nt_ch_stunt = c(99))) %>%
  set_value_labels(nt_ch_stunt = c("Yes" = 1, "No"=0  )) %>%
  set_variable_labels(nt_ch_stunt = "Stunted child under 5 years")

# //Mean haz
PRdata <- PRdata %>%
  mutate(haz = case_when(hv103 ==1 & hc70 < 996 ~ hc70/100)) 
  PRdata$nt_ch_mean_haz <- matrixStats::weightedMean(PRdata$haz, PRdata$wt, idxs = NULL, na.rm = TRUE) 

# //Severely wasted 
PRdata <- PRdata %>%
  mutate(nt_ch_sev_wast =
           case_when(
             hv103==1 &  hc72< -300  ~ 1 ,
             hv103==1 &  hc72>= -300 & hc72<9996 ~ 0 ,
             hc72>=9996 ~ 99)) %>%
  replace_with_na(replace = list(nt_ch_sev_wast = c(99))) %>%
  set_value_labels(nt_ch_sev_wast = c("Yes" = 1, "No"=0  )) %>%
  set_variable_labels(nt_ch_sev_wast = "Severely wasted child under 5 years")

# //Wasted
PRdata <- PRdata %>%
  mutate(nt_ch_wast =
           case_when(
             hv103==1 &  hc72< -200  ~ 1 ,
             hv103==1 &  hc72>= -200 & hc72<9996~ 0 ,
             hc72>=9996 ~ 99)) %>%
  replace_with_na(replace = list(nt_ch_wast = c(99))) %>%
  set_value_labels(nt_ch_wast = c("Yes" = 1, "No"=0  )) %>%
  set_variable_labels(nt_ch_wast = "Wasted child under 5 years")

# //Overweight for height
PRdata <- PRdata %>%
  mutate(nt_ch_ovwt_ht =
           case_when(
             hv103==1 &  hc72> 200 & hc72<9996 ~ 1 ,
             hv103==1 &  hc72<= 200 & hc72<9996 ~ 0 ,
             hc72>=9996 ~ 99)) %>%
  replace_with_na(replace = list(nt_ch_ovwt_ht = c(99))) %>%
  set_value_labels(nt_ch_ovwt_ht = c("Yes" = 1, "No"=0  )) %>%
  set_variable_labels(nt_ch_ovwt_ht = "Overweight for height child under 5 years")

# //Mean whz
PRdata <- PRdata %>%
  mutate(whz = case_when(hv103 ==1 & hc72<996 ~ hc72/100)) 
PRdata$nt_ch_mean_whz <- matrixStats::weightedMean(PRdata$whz, PRdata$wt, idxs = NULL, na.rm = TRUE) 

# //Severely underweight
PRdata <- PRdata %>%
  mutate(nt_ch_sev_underwt =
           case_when(
             hv103==1 &  hc71< -300  ~ 1 ,
             hv103==1 &  hc71>= -300 & hc71<9996 ~ 0 ,
             hc71>=9996 ~ 99)) %>%
  replace_with_na(replace = list(nt_ch_sev_underwt = c(99))) %>%
  set_value_labels(nt_ch_sev_underwt = c("Yes" = 1, "No"=0  )) %>%
  set_variable_labels(nt_ch_sev_underwt = "Severely underweight child under 5 years")

# //Underweight
PRdata <- PRdata %>%
  mutate(nt_ch_underwt =
           case_when(
             hv103==1 &  hc71< -200  ~ 1 ,
             hv103==1 &  hc71>= -200  & hc71<9996 ~ 0 ,
             hc71>=9996 ~ 99)) %>%
  replace_with_na(replace = list(nt_ch_underwt = c(99))) %>%
  set_value_labels(nt_ch_underwt = c("Yes" = 1, "No"=0  )) %>%
  set_variable_labels(nt_ch_underwt = "Underweight child under 5 years")
# 
# //Overweight for age
PRdata <- PRdata %>%
  mutate(nt_ch_ovwt_age =
           case_when(
             hv103==1 &  hc71> 200 & hc71<9996 ~ 1 ,
             hv103==1 &  hc71<= 200  & hc71<9996 ~ 0 ,
             hc71>=9996 ~ 99)) %>%
  replace_with_na(replace = list(nt_ch_ovwt_age = c(99))) %>%
  set_value_labels(nt_ch_ovwt_age = c("Yes" = 1, "No"=0  )) %>%
  set_variable_labels(nt_ch_ovwt_age = "Overweight for age child under 5 years")

# //Mean waz
PRdata <- PRdata %>%
  mutate(waz = case_when(hv103 ==1 & hc71<996 ~ hc71/100)) 
PRdata$nt_ch_mean_waz <- matrixStats::weightedMean(PRdata$waz, PRdata$wt, idxs = NULL, na.rm = TRUE) 

# *** Anemia indicators ***

# //Any anemia
PRdata <- PRdata %>%
  mutate(nt_ch_any_anem =
           case_when(
             hv103==1 & hc1>5 & hc1<60 & hc56<110 ~ 1 ,
             hv103==1 & hc1>5 & hc1<60 & hc56>=110 ~ 0)) %>%
  set_value_labels(nt_ch_any_anem = c("Yes" = 1, "No"=0  )) %>%
  set_variable_labels(nt_ch_any_anem = "Any anemia - child 6-59 months")

# //Mild anemia
PRdata <- PRdata %>%
  mutate(nt_ch_mild_anem =
           case_when(
             hv103==1 & hc1>5 & hc1<60 & hc56>99 & hc56<110 ~ 1 ,
             hv103==1 & hc1>5 & hc1<60 & hc56<=99 | hc56>=110 ~ 0)) %>%
  set_value_labels(nt_ch_mild_anem = c("Yes" = 1, "No"=0  )) %>%
  set_variable_labels(nt_ch_mild_anem = "Mild anemia - child 6-59 months")

# //Moderate anemia
PRdata <- PRdata %>%
  mutate(nt_ch_mod_anem =
           case_when(
             hv103==1 & hc1>5 & hc1<60 & hc56>69 & hc56<100 ~ 1 ,
             hv103==1 & hc1>5 & hc1<60 & hc56<=69 | hc56>=100 ~ 0)) %>%
  set_value_labels(nt_ch_mod_anem = c("Yes" = 1, "No"=0  )) %>%
  set_variable_labels(nt_ch_mod_anem = "Moderate anemia - child 6-59 months")

# //Severe anemia
PRdata <- PRdata %>%
  mutate(nt_ch_sev_anem =
           case_when(
             hv103==1 & hc1>5 & hc1<60 & hc56<70 ~ 1 ,
             hv103==1 & hc1>5 & hc1<60 & hc56>=70 ~ 0)) %>%
  set_value_labels(nt_ch_sev_anem = c("Yes" = 1, "No"=0  )) %>%
  set_variable_labels(nt_ch_sev_anem = "Severe anemia - child 6-59 months")

  PRdata <- PRdata %>%
    dplyr::select(survey, hhid, hvidx, hv001, hv002, hvidx, wt, hv021, hv023, hv024, hv025, hv103, hc70, nt_ch_sev_stunt, nt_ch_stunt, haz, nt_ch_mean_haz, hc72, nt_ch_sev_wast, nt_ch_wast, nt_ch_ovwt_ht, whz, nt_ch_mean_whz, hc71, nt_ch_sev_underwt, nt_ch_underwt, nt_ch_ovwt_age, waz, nt_ch_mean_waz, hc1, nt_ch_any_anem, hc56, nt_ch_mild_anem, nt_ch_mod_anem, nt_ch_sev_anem) %>%
    dplyr::mutate(across(where(is.labelled), as_factor)) %>%
    dplyr::mutate_if(is.factor, as.character) %>%
    dplyr::mutate_all(as.character) %>%
    distinct()

chn_indicators <- bind_rows(chn_indicators, PRdata)

}

In [0]:
# chn_indicators1 <- chn_indicators %>%
#   dplyr::select(survey, hhid, hvidx, hv001, hv002, wt, hv021, hv023, hv024, hv025, hv103, hc70, nt_ch_sev_stunt, nt_ch_stunt, haz, nt_ch_mean_haz, hc72, nt_ch_sev_wast, nt_ch_wast, nt_ch_ovwt_ht, whz, nt_ch_mean_whz, hc71, nt_ch_sev_underwt, nt_ch_underwt, nt_ch_ovwt_age, waz, nt_ch_mean_waz, hc1, nt_ch_any_anem, hc56, nt_ch_mild_anem, nt_ch_mod_anem, nt_ch_sev_anem) %>%
#   dplyr::mutate(across(where(is.labelled), as_factor)) %>%
#   dplyr::mutate_if(is.factor, as.character) %>%
#   distinct()

chn_indicators1 <- chn_indicators %>%
  rename(v001 = hv001,
         v002 = hv002) %>%
  base::merge(pts_comb1, by=c("survey", "v001"), all.x=TRUE) %>%
  distinct()

display(chn_indicators1)

In [0]:
# THESE HAVE BOGUS GPS COORDINATES, CAN FIX MOST OF THEM BY HAND USING THE SURVEY STRATIFICATION VARIABLES
display(
  chn_indicators1 %>%
  filter(is.na(admin1)) %>%
  group_by(survey) %>%
  dplyr::summarize(count = n())
  )

In [0]:
chn_indicators1_sdf <- chn_indicators1 %>%
  as.DataFrame()

In [0]:
delta_path = "abfss://dhs-data@idmdpdls01.dfs.core.windows.net/dhs_indicators/dhs_nt_ch_nut.delta"
dbutils.fs.rm(delta_path, "true")
write.df(chn_indicators1_sdf, source = 'delta', path = delta_path, mode = 'overwrite')

catalog = "idm_dhs"
schema = "dhs_indicators"
table_name = "dhs_nt_ch_nut"
file_path = delta_path

# SparkR::sql(paste0("CREATE CATALOG IF NOT EXISTS ", catalog))
SparkR::sql(paste0("CREATE SCHEMA IF NOT EXISTS ", catalog, ".", schema))
SparkR::sql(paste0("DROP TABLE IF EXISTS ", catalog, ".", schema, ".", table_name))
SparkR::createTable(paste0(catalog, ".", schema, ".", table_name), path=file_path, source="delta")
print(paste0(catalog, ".", schema, ".", table_name))